In [1]:
%pylab inline

import pandas as pd
import os

from collections import OrderedDict

Populating the interactive namespace from numpy and matplotlib


In [2]:
rdir = os.path.expandvars("$HOME/Research/photometric-ew")

sdss_res = loadtxt("{}/data/SDSS_filters.txt".format(rdir))
sdss_res = [
    sdss_res[:47],
    sdss_res[47:47+89],
    sdss_res[47+89:47+89+75],
    sdss_res[47+89+75:47+89+75+89],
    sdss_res[47+89+75+89:]
]
jpas = open("{}/data/JPAS_filters.txt".format(rdir)).readlines()

mask = [False if item.startswith("#") else True for item in jpas]
jpas_res = []
for i in xrange(mask.count(False)):
    try:
        ini = mask.index(False); mask.remove(False)
        fin = mask.index(False)
        
        jpas_res.append(np.array([eval(item.replace("  ", ",")) for item in jpas[ini+1+i:fin+1+i]]))
        
    except ValueError:
        jpas_res.append(np.array([eval(item.replace("  ", ",")) for item in jpas[ini+1+i:]]))

In [3]:
sdss_filters = ["u", "g", "r", "i", "z"]
jpas_filters = ["JP{0:02d}".format(1+i) for i in xrange(len(jpas_res))]

In [4]:
surveys = []
filters = []
for j, filter_name in enumerate(sdss_filters):
    filters += [filter_name]*sdss_res[j].shape[0]
surveys += ["SDSS"]*len(filters)
for j, filter_name in enumerate(jpas_filters):
    filters += [filter_name]*jpas_res[j].shape[0]
surveys += ["J-PAS"]*(len(filters)-len(surveys))

responses = vstack(sdss_res+jpas_res)

In [5]:
data = OrderedDict(
    zip(
        ("survey", "filter", "wavelength", "response"),
        (surveys, filters, responses[:, 0], responses[:, 1])
    )
)
df = pd.DataFrame(data)

In [6]:
df.head()

,survey,filter,wavelength,response
0,SDSS,u,2980.0,0.0000
1,SDSS,u,3005.0,0.0001
2,SDSS,u,3030.0,0.0005
3,SDSS,u,3055.0,0.0013
4,SDSS,u,3080.0,0.0026


In [7]:
df.to_csv(path_or_buf="../data/filters.csv", index=False)